# Proyecto SQL

## 1. Objetivos del estudio

Durante la pandemia de coronavirus, más personas se quedaron en casa leyendo libros, lo que atrajo a startups que desarrollaron aplicaciones para amantes de los libros. El objetivo general de este proyecto es analizar un grupo bases de datos de uno de estos servicios para generar una propuesta de valor para un nuevo producto.

Los objetivos específicos de este estudio son:
* Analizar el mercado editorial moderno
* Evaluar la calidad y popularidad de los libros
* Identificar editoriales relevantes
* Descubrir autores de alta calidad
* Analizar el comportamiento de usuarios activos

## 2. Conectarse a la base de datos

In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## 3. Estudiar las tablas

In [2]:
query = "SELECT * FROM books LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = "SELECT * FROM authors LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = "SELECT * FROM publishers LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = "SELECT * FROM ratings LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = "SELECT * FROM reviews LIMIT 5"
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## 4. Realizar una consulta SQL para cada una de las tareas

In [7]:
# Encuentra el número de libros publicados después del 1 de enero de 2000
query = "SELECT COUNT(title) AS cnt_books FROM books WHERE publication_date > '2000-01-01'"
pd.io.sql.read_sql(query, con = engine)

,cnt_books
0,819


Teniendo en cuenta los resultados, podemos decir que 819 representa el mercado moderno editorial, ya que casi toda nuestra base contiene títulos publicados desde el 2000. Esto también nos quiere decir que la plataforma se enfoca en contenido relativamente reciente (últimas 2 décadas). Para una aplicación de libros, esto es estratégicamente inteligente,  ya que los libros más nuevos suelen tener más demanda y menos competencia con bibliotecas gratuitas.

In [8]:
# Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro
# Unimos primero las tablas review y rating por username y book_id, luego hacemos la consulta
query = "WITH reviews_ratings AS (SELECT reviews.book_id, reviews.username, reviews.text, ratings.rating FROM reviews INNER JOIN ratings ON ratings.username = reviews.username AND ratings.book_id = reviews.book_id) SELECT COUNT(*) as num_reviews, AVG(rating) AS avg_rating, book_id FROM reviews_ratings GROUP BY book_id"
pd.io.sql.read_sql(query, con = engine)

,num_reviews,avg_rating,book_id
0,2,4.500000,652
1,2,4.500000,273
2,5,4.600000,51
3,2,4.000000,951
4,4,4.500000,839
...,...,...,...
989,4,4.500000,64
990,2,5.000000,55
991,3,3.333333,148
992,2,3.500000,790


El total de libros publicados con reseñas y calificaciones fue de 994. Las calificaciones promedio van desde 3 hasta 5 y el número de reseñas varía de 2 a 5 por libro. Esto nos puede indicar que los usuarios de la plataforma tienen un egagement moderado pero consistente y que las calificaciones son buenas en general, lo que nos lleva a pensar que tienen una buena curaduría al momento de seleccionar los libros y autores.

In [9]:
# Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas
# Unimos las tablas books con publisher por publisher_id, luego hacemos la consulta
query = "WITH books_publishers AS (SELECT books.title, books.num_pages, publishers.publisher_id, publishers.publisher FROM books INNER JOIN publishers ON publishers.publisher_id = books.publisher_id) SELECT COUNT(title) AS num_books, publisher, publisher_id FROM books_publishers WHERE num_pages > 50 GROUP BY publisher, publisher_id ORDER BY num_books DESC"
pd.io.sql.read_sql(query, con = engine)

,num_books,publisher,publisher_id
0,42,Penguin Books,212
1,31,Vintage,309
2,25,Grand Central Publishing,116
3,24,Penguin Classics,217
4,19,Ballantine Books,33
...,...,...,...
329,1,Ballantine Books (NY),34
330,1,Plaza y Janés,225
331,1,HarperCollinsPublishers,138
332,1,Random House: Modern Library,245


Penguin Books es la editorial líder con 42 publicaciones. Le siguen Vintage con 31 libros y Grand Central Publishing con 25 libros. Con ello, podemos decir que Penguin domina el mercado editoria, ya que es la más grande firma del mercado.

Por otra parte, notamos que el total de editoriales publicadas es de 334. Lo que nos lleva a pensar que tienen gran variedad de editoriales y esto permite conocer títulos muy variados que no se centran únicamente en un tipo de lector.

In [10]:
# Identifica al autor que tiene la más alta calificación promedio del libro
query = "WITH books_authors_rating AS (SELECT books.title, authors.author, ratings.rating FROM books INNER JOIN authors ON authors.author_id = books.author_id INNER JOIN ratings ON ratings.book_id = books.book_id) SELECT author, AVG(rating) AS avg_rating FROM books_authors_rating GROUP BY author ORDER BY avg_rating DESC LIMIT 30"
pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,Nelson DeMille,5.000000
1,Charles Dickens,5.000000
2,Jennifer Crusie,5.000000
3,Neil Gaiman/Andy Kubert/Richard Isanove/Peter ...,5.000000
4,Jon Kabat-Zinn,5.000000
5,Nikolai Gogol/Robert A. Maguire/Zlatko Crnković,5.000000
6,Carl Sagan/Ann Druyan,5.000000
7,Pat Frank,5.000000
8,James Patterson/مریم کاظمی‌تبار,5.000000
9,Barbara Taylor Bradford,5.000000


Hay muchos autores (29) con calificación perfecta (5.0), incluyendo:
* Nelson DeMille
* Charles Dickens  
* Jennifer Crusie
* Neil Gaiman...

Sin embargo, esto no nos dice mucho sobre la cantidad de calificaciones que reciben, por lo que el promedio puede variar por la cantidad de puntuaciones más que por la puntuación en sí. Esto no nos da una mirada objetiva para saber si son verdaderamente los más populares o no.

In [12]:
query = """
WITH books_authors_rating AS (
    SELECT 
        authors.author,
        ratings.rating
    FROM books
    INNER JOIN authors ON authors.author_id = books.author_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
)
SELECT 
    author,
    AVG(rating) AS avg_rating,
    COUNT(rating) AS rating_count
FROM books_authors_rating
GROUP BY author
HAVING COUNT(rating) >= 50
ORDER BY avg_rating DESC
LIMIT 1;
"""

pd.read_sql(query, con=engine)


,author,avg_rating,rating_count
0,Diana Gabaldon,4.3,50


In [11]:
# Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros
# Primero necesitamos identificar qué usuarios han calificado más de 50 libros
query = "SELECT username, COUNT(*) as num_ratings FROM ratings GROUP BY username HAVING COUNT(*) > 50"
# Luego, para esos usuarios, contamos cuántas reseñas de texto han escrito
query = "WITH active_users AS (SELECT username FROM ratings GROUP BY username HAVING COUNT(*) > 50) SELECT AVG(review_count) as avg_reviews FROM (SELECT active_users.username, COUNT(reviews.review_id) as review_count FROM active_users LEFT JOIN reviews ON active_users.username = reviews.username GROUP BY active_users.username) user_reviews"
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.333333


Los usuarios que califican más de 50 libros (usuarios muy activos) escriben en promedio 24,33 reseñas de texto. Esto significa que aproximadamente la mitad de los libros que califican también los reseñan con texto. Es un nivel de compromiso bastante alto, ya que no solo califican los libros, sino que se toman el tiempo de escribir reseñas detalladas. Esto se puede deber a que los usuarios representan una audiencia más valiosa, es decir, son lectores ávidos y buscan contenido de calidad.

## Conclusiones adicionales

* Los libros con más reseñas no siempre tienen mejores calificaciones, así que es importante destacar la diferencia entre reseñas y puntuaciones de los lectores en cuanto a títulos y autores.
* Los usuarios que escriben más de 24 reseñas, son el activo más valioso para la plataforma, así que es importante centrar la atención en la calidad de la información que facilitan estos usuarios.
* Se puede incentivar a los usuarios casuales a ser más activos, lo que proporcionaría información de calidad paran centrar la atención en esos títulos con mayor popularidad.
* Se puede negociar con partners estratégicos como Penguin, Vintage y Grand Central para tener mejores precios y aumentar la demanda. Así como, usarlos de plataforma para promocionar la aplicación con lectores que los prefieran.